In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("/kaggle/input/bigmart-sales-data/Train.csv")
df.head()

In [5]:
df.shape

In [6]:
df.info()

In [7]:
df.isnull().sum()

In [8]:
mean = df['Item_Weight'].mean()
print(mean)

In [9]:
df['Item_Weight'].fillna(mean,inplace=True)

In [10]:
df.head()

In [11]:
df.isnull().sum()

In [12]:
from scipy.stats import mode
# filling the missing values in "Outlet_Size" column with Mode
os_mode = df.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc=(lambda x: (x.mode()[0])))

In [13]:
os_mode

In [14]:
missing_or_not = df['Outlet_Size'].isnull()

In [15]:
missing_or_not

In [16]:
# df.loc[missing_or_not,'Outlet_Size'] = df.loc[missing_or_not,'Outlet_Size'].apply(lambda x:mode[x])

In [17]:
df.loc[missing_or_not,'Outlet_Size'] = df.loc[missing_or_not,'Outlet_Type'].apply(lambda x: os_mode[x])

In [18]:
df.isnull().sum()

## Data Analysis

In [19]:
df.describe()

In [20]:
plt.figure(figsize=(6,6))
sns.distplot(df['Item_Weight'])
plt.show()

In [21]:
plt.figure(figsize=(6,6))
sns.distplot(df['Item_Visibility'])
plt.show()

In [22]:
plt.figure(figsize=(6,6))
sns.distplot(df['Item_MRP'])
plt.show()

In [23]:
plt.figure(figsize=(6,6))
sns.distplot(df['Item_Outlet_Sales'])
plt.show()

In [24]:
plt.figure(figsize=(6,6))
sns.countplot(x='Outlet_Establishment_Year', data=df)
plt.show()

In [25]:
plt.figure(figsize=(6,6))
sns.countplot(x='Item_Fat_Content', data=df)
plt.show()

In [26]:
df['Item_Fat_Content'].value_counts()

In [27]:
df.replace({
    'Item_Fat_Content': {
        'low fat':'Low Fat',
        'LF':'Low Fat', 
        'reg':'Regular'
    }
}, inplace=True)

In [28]:
df['Item_Fat_Content'].value_counts()

In [29]:
plt.figure(figsize=(30,6))
sns.countplot(x='Item_Type', data=df)
plt.show()

In [30]:
plt.figure(figsize=(6,6))
sns.countplot(x='Outlet_Size', data=df)
plt.show()

In [31]:
X = df.drop('Item_Outlet_Sales',axis=1)
y = df.Item_Outlet_Sales

In [32]:
cols = ['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Size','Outlet_Location_Type','Outlet_Type'] 

X = pd.get_dummies(X, columns = cols)

In [35]:
# for i in cols:
#     count = X[i].value_counts()
#     print("Column name is:",i,"and its value is:\n",count)

In [34]:
X.head()

In [36]:
X.shape

In [37]:
from sklearn.preprocessing import LabelEncoder

In [38]:
le = LabelEncoder()
X['Item_Identifier'] = le.fit_transform(X['Item_Identifier'])

In [39]:
X.head()

In [40]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [41]:
X_train.shape

In [42]:
X_test.shape

In [43]:
from xgboost import XGBRegressor

In [44]:
xgbr = XGBRegressor()
xgbr.fit(X_train,y_train)

In [45]:
xgbr.score(X_train,y_train)

In [46]:
xgbr.score(X_test,y_test)

In [47]:
# Alternative code
from sklearn.metrics import r2_score
predict = xgbr.predict(X_train)
r2 = r2_score(y_train,predict)
r2